In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

In [2]:
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 'TC_AUC',0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3,True

"""=====================================================================================================================
1 读取数据
"""
path = "E:/data2/"
op_train = pd.read_csv(path+"operation_train_new.csv")
trans_train = pd.read_csv(path+"transaction_train_new.csv")

op_test = pd.read_csv(path+"operation_round1_new.csv")
trans_test = pd.read_csv(path+"transaction_round1_new.csv")
y = pd.read_csv(path+"tag_train_new.csv")

sub = pd.read_csv(path+"sub.csv")


def get_feature(op,trans,label): # 分别读取operation, transaction, y
    for feature in op.columns[2:]:
        label = label.merge(op.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')#先把y附加进dataframe, 对UID进行分组，分别统计各个UID出现的数量
        label =label.merge(op.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')#对UID进行分组，分别统计各个UID的不唯一的数量
    
    for feature in trans.columns[2:]:
        if trans_train[feature].dtype == 'object':# 判断是否是可以统计的属性
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
        else:
            label =label.merge(trans.groupby(['UID'])[feature].count().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].nunique().reset_index(),on='UID',how='left')
            label =label.merge(trans.groupby(['UID'])[feature].max().reset_index(),on='UID',how='left') #附加每个属性最大的
            label =label.merge(trans.groupby(['UID'])[feature].min().reset_index(),on='UID',how='left')#附加每个属性最小的
            label =label.merge(trans.groupby(['UID'])[feature].sum().reset_index(),on='UID',how='left')#附加求和
            label =label.merge(trans.groupby(['UID'])[feature].mean().reset_index(),on='UID',how='left')#附加均值
            label =label.merge(trans.groupby(['UID'])[feature].std().reset_index(),on='UID',how='left')#附加方差
    return label

C:\Users\Simon\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
sheng1995
sheng1995
sha1:d447f8a39ea5:c2c014b3900bf9e905227d0917eff581788cd9c0
        vim /home/run/.jupyter/jupyter_notebook_config.
        sha1:9576280022ee:fdcfd2a97f4d26e8d445551b5b358d2906027c27
                sha1:e051e4ddd695:ebfa551bf1ad73ad8083bba7cabc0b9f98d8f69c
                        sha1:d2291cd5a11e:42d59346c1eab9cc5571d4ca8b4f3709cbdcebad
                                sha1:fbd01786a0a2:265c82ee42afcc87845e5d4ecb3cc09713e77da3

In [3]:
train = get_feature(op_train,trans_train,y).fillna(-1)#用-1填充空格
test = get_feature(op_test,trans_test,sub).fillna(-1

train = train.drop(['UID','Tag'],axis = 1).fillna(-1)
label = y['Tag']

test_id = test['UID']
test = test.drop(['UID','Tag'],axis = 1).fillna(-1)


lgb_model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=100, reg_alpha=3, reg_lambda=5, max_depth=-1,
    n_estimators=5000, objective='binary', subsample=0.9, colsample_bytree=0.77, subsample_freq=1, learning_rate=0.01,
    random_state=1000, n_jobs=16, min_child_weight=4, min_child_samples=5, min_split_gain=0)# learning_rate=0.05


skf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)##5折价交差验证
best_score = []

oof_preds = np.zeros(train.shape[0])#做一个与训练集形状相同的矩阵
sub_preds = np.zeros(test_id.shape[0])#做一个与测试集Id形状相同的矩阵

for index, (train_index, test_index) in enumerate(skf.split(train, label)):#用5折交叉验证切分train 和 label, 用enumerate给每一组数据加一个索引
    lgb_model.fit(train.iloc[train_index], label.iloc[train_index], verbose=50,
                  eval_set=[(train.iloc[train_index], label.iloc[train_index]),
                            (train.iloc[test_index], label.iloc[test_index])], early_stopping_rounds=60)#early_stopping_rounds=30
    best_score.append(lgb_model.best_score_['valid_1']['binary_logloss'])
    print(best_score)
    oof_preds[test_index] = lgb_model.predict_proba(train.iloc[test_index], num_iteration=lgb_model.best_iteration_)[:,1]

    test_pred = lgb_model.predict_proba(test, num_iteration=lgb_model.best_iteration_)[:, 1]#取预测记过的第二列（预测为黑样本的概率）
    sub_preds += test_pred / 5 #5折交叉，每次取1/5
    #print('test mean:', test_pred.mean())
    #predict_result['predicted_score'] = predict_result['predicted_score'] + test_pred

m = tpr_weight_funtion(y_predict=oof_preds,y_true=label)
print(m[1])
sub = pd.read_csv(path+"sub.csv")
sub['Tag'] = sub_preds
#sub.to_csv(path+"lgb_baseline_%s.csv"%str(m),index=False)

Training until validation scores don't improve for 60 rounds.
[50]	valid_0's binary_logloss: 0.2596	valid_1's binary_logloss: 0.263622
[100]	valid_0's binary_logloss: 0.202916	valid_1's binary_logloss: 0.209875
[150]	valid_0's binary_logloss: 0.170264	valid_1's binary_logloss: 0.180227
[200]	valid_0's binary_logloss: 0.149662	valid_1's binary_logloss: 0.162186
[250]	valid_0's binary_logloss: 0.135586	valid_1's binary_logloss: 0.150695
[300]	valid_0's binary_logloss: 0.125179	valid_1's binary_logloss: 0.142839
[350]	valid_0's binary_logloss: 0.117272	valid_1's binary_logloss: 0.13734
[400]	valid_0's binary_logloss: 0.110911	valid_1's binary_logloss: 0.133276
[450]	valid_0's binary_logloss: 0.105483	valid_1's binary_logloss: 0.130203
[500]	valid_0's binary_logloss: 0.100858	valid_1's binary_logloss: 0.127855
[550]	valid_0's binary_logloss: 0.0968026	valid_1's binary_logloss: 0.126045
[600]	valid_0's binary_logloss: 0.0931938	valid_1's binary_logloss: 0.124644
[650]	valid_0's binary_loglo

[1250]	valid_0's binary_logloss: 0.0643115	valid_1's binary_logloss: 0.129982
[1300]	valid_0's binary_logloss: 0.0630495	valid_1's binary_logloss: 0.129805
[1350]	valid_0's binary_logloss: 0.0618247	valid_1's binary_logloss: 0.129638
[1400]	valid_0's binary_logloss: 0.0607081	valid_1's binary_logloss: 0.129575
[1450]	valid_0's binary_logloss: 0.0596186	valid_1's binary_logloss: 0.129545
[1500]	valid_0's binary_logloss: 0.05859	valid_1's binary_logloss: 0.129564
Early stopping, best iteration is:
[1470]	valid_0's binary_logloss: 0.0591954	valid_1's binary_logloss: 0.129523
[0.11575920800973363, 0.1154264580259414, 0.12952282230758125]
Training until validation scores don't improve for 60 rounds.
[50]	valid_0's binary_logloss: 0.258984	valid_1's binary_logloss: 0.266155
[100]	valid_0's binary_logloss: 0.202218	valid_1's binary_logloss: 0.213946
[150]	valid_0's binary_logloss: 0.169567	valid_1's binary_logloss: 0.185231
[200]	valid_0's binary_logloss: 0.148657	valid_1's binary_logloss: 0.

In [4]:
 lgb_model.feature_importance()

AttributeError: 'LGBMClassifier' object has no attribute 'feature_importance'

In [ ]:
train[1:5][-5:-1]

In [5]:
booster = lgb_model.booster_
importance = booster.feature_importance(importance_type='split')

In [6]:
importance

array([3873, 4564, 2456,  351,  914, 5105,  156,  951, 1374,  597,  574,
        451, 3941,  324, 1546,  366, 3157, 1154, 1138,   50, 3528,  652,
       4478, 1453, 1636, 2479,  983,  131, 7030, 1150, 5559,  462,  445,
       2360,  186,   63,  335,  983, 3878, 2762, 6151, 6191, 4945,   81,
       1235,   23, 1587, 3533, 4178, 6155, 5190, 5619,    4,  846,    3,
       1192,  629,  396,    0,    0,    0,  307,  699,  294,  693,  152,
       1159,  249,  403,  263,  367,  100,  162,  195,  594,  100,   83,
        617,    0,  439,  156,    7, 1156,  183,  187, 2009,  401,  117,
         94,  108,   62, 1811,  191, 1167,  428,  542,  178, 1509, 1451,
        624,  473,  377,  100,   30,   50,   75,  522,   94,    8,   22,
        788])